## Preprocessing

In [60]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [61]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df=application_df.drop(columns=['EIN','NAME'],axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [62]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [ ]:
app_type=application_df['APPLICATION_TYPE'].value_counts()
app_type


In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace=[x for x in app_type.keys() if (app_type[x]<200)]

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

In [ ]:
class_type=application_df['CLASSIFICATION'].value_counts()
class_type 

In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_type[class_type>1]

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace=[x for x in class_type.keys() if (class_type[x]<1000)]

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()


In [83]:
m=pd.get_dummies(application_df)
m

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
X=pd.get_dummies(application_df.drop(columns='STATUS',axis=1))
X

In [85]:
# Split our preprocessed data into our features and target arrays
y=application_df['STATUS']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [86]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [94]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_dim=X.shape[1]

nn_model = tf.keras.models.Sequential()
dense = tf.keras.layers.Dense(input_dim=input_dim, units=80, activation="relu")
nn_model.add(dense)

# First hidden layer
dense_1 = tf.keras.layers.Dense(units=30, activation="relu")
nn_model.add(dense_1)

# Second hidden layer
#  YOUR CODE GOES HERE

# Output layer
dense_2 = tf.keras.layers.Dense(units=1, activation="sigmoid")
nn_model.add(dense_2)

# Check the structure of the model
nn_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 80)                3520      
                                                                 
 dense_8 (Dense)             (None, 30)                2430      
                                                                 
 dense_9 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [95]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [103]:
X_train_scaled.shape
y_train.shape

(25724,)

In [106]:
from keras.callbacks import Callback
model_callback = tf.keras.callbacks.ModelCheckpoint('weights{epoch:02d}.h5', 
                                     save_weights_only=True, save_freq=5)


In [107]:
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=20,callbacks=[model_callback])

Epoch 1/20
804/804 [==============================] - 3s 4ms/step - loss: 0.0020 - accuracy: 0.9998
Epoch 2/20
804/804 [==============================] - 4s 5ms/step - loss: 0.0014 - accuracy: 0.9999
Epoch 3/20
804/804 [==============================] - 3s 3ms/step - loss: 6.2899e-04 - accuracy: 0.9998
Epoch 4/20
804/804 [==============================] - 3s 3ms/step - loss: 6.1705e-04 - accuracy: 0.9998
Epoch 5/20
804/804 [==============================] - 3s 3ms/step - loss: 4.7899e-04 - accuracy: 0.9999
Epoch 6/20
804/804 [==============================] - 4s 5ms/step - loss: 6.3093e-04 - accuracy: 0.9999
Epoch 7/20
804/804 [==============================] - 3s 4ms/step - loss: 5.6707e-04 - accuracy: 0.9999
Epoch 8/20
804/804 [==============================] - 3s 3ms/step - loss: 5.5066e-04 - accuracy: 0.9999
Epoch 9/20
804/804 [==============================] - 3s 3ms/step - loss: 6.1485e-04 - accuracy: 0.9999
Epoch 10/20
804/804 [==============================] - 3s 4ms/step - los

In [111]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.0090 - accuracy: 0.9998 - 463ms/epoch - 2ms/step
Loss: 0.009002306498587132, Accuracy: 0.9997667670249939


In [110]:
# Export our model to HDF5 file
nn_model.save("AlphabetSoupCharity.h5")